# Итоговое задание №2. Прогнозирование конечных свойств новых материалов (композиционных материалов)
На входе имеются данные о начальных свойствах компонентов композиционных материалов (количество связующего, наполнителя, температурный режим отверждения и т.д.). На выходе необходимо спрогнозировать ряд конечных свойств получаемых композиционных материалов. 

Предсказание (из семинара 22): 
* модуль упругости при растяжении
* прочности при растяжении
* cоотношение матрица-наполнитель

Предсказание (из итогового вебинара 26.11.2022):
* модуль упругости при растяжении
* прочности при растяжении

загружаем датасет

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# считывание данных из файла
def load_simple_df(full_nm_str):
    sdf = pd.read_csv('../data/interim/'+full_nm_str+'.csv', sep=';',index_col=0) # Загрузка и создание датафрейма из csv файла
    return sdf

#считывание всех наборов для одного из целевых показателей
def load_data(nm_str):
    x_train = load_simple_df('X_train_'+nm_str)
    y_train = load_simple_df('y_train_'+nm_str)
    x_test = load_simple_df('X_test_'+nm_str)
    y_test=load_simple_df('y_test_'+nm_str)
    x_val=load_simple_df('X_val_'+nm_str)
    y_val=load_simple_df('y_val_'+nm_str)
    return (x_train, y_train, x_test, y_test, x_val, y_val)

In [19]:
(x_train_df, y_train_df, x_test_df, y_test_df, x_val_df, y_val_df) = load_data('elastic')

In [20]:
x_train_df.head()

,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,1862.432065,1428.870308,125.557042,24.023624,317.571591,293.085220,220.481828,0.0,0.310815,70.157259
1,2019.574351,566.801799,108.053040,24.317107,327.219188,443.686026,219.198882,0.0,7.303776,52.922258
2,2158.794836,923.345018,155.224211,25.397131,262.325518,315.195712,207.662030,90.0,3.198546,53.758941
3,1954.917354,703.754668,125.799949,25.001898,269.251889,538.997427,280.854848,90.0,6.731857,74.109337
4,1919.333909,876.772075,107.850574,24.152744,253.127721,162.952788,322.266101,90.0,9.243225,63.650692


In [21]:
y_train_df.head()

,"Модуль упругости при растяжении, ГПа"
0,81.594750
1,80.114266
2,73.495350
3,67.651769
4,75.966376


In [25]:
# Рассматриваемые модели модели используем методы:
regression_models = [LinearRegression(),
            KNeighborsRegressor(),
            SVR(kernel='linear'),
            AdaBoostRegressor(),
            BaggingRegressor(),
            ExtraTreesRegressor(),
            RandomForestRegressor(),
            GradientBoostingRegressor(),
            DecisionTreeRegressor(max_depth=5)
            ]

In [26]:
print(f"x(train, test): {x_train_df.shape}, {x_test_df.shape}")
print(f"y(train, test): {y_train_df.shape}, {y_test_df.shape}")

x(train, test): (501, 10), (307, 10)
y(train, test): (501, 1), (307, 1)


In [27]:
#создаем временные структуры
test_models = pd.DataFrame()
tmp = {}
#для каждой модели из списка
for model in regression_models:
    #получаем имя модели
    m = str(model)
    tmp['Model'] = m[:m.index('(')]

    #для каждого столбцам результирующего набора
    for output in ('Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Соотношение матрица-наполнитель'):
        #обучаем модель
        model.fit(x_train_df, y_train_df) 
        #вычисляем коэффициент детерминации
        if output == 'Модуль упругости при растяжении, ГПа':
            nm='elastic'
        elif output == 'Прочность при растяжении, МПа':
            nm='strength'
        elif output == 'Соотношение матрица-наполнитель':
            nm='matrix'

        tmp['R2_%s'%nm] = r2_score(y_test_df, model.predict(x_test_df))
        tmp['MAE_%s'%nm] = mean_absolute_error(y_test_df, model.predict(x_test_df))
        tmp['MSE_%s'%nm] = mean_squared_error(y_test_df, model.predict(x_test_df))
    
    #записываем данные и итоговый DataFrame
    test_models = test_models.append([tmp])
#делаем индекс по названию модели
test_models.set_index('Model', inplace=True)

test_models

,R2_elastic,MAE_elastic,MSE_elastic,R2_strength,MAE_strength,MSE_strength,R2_matrix,MAE_matrix,MSE_matrix
Model,,,,,,,,,
LinearRegression,-0.003139,2.647692,10.804302,-0.003139,2.647692,10.804302,-0.003139,2.647692,10.804302
KNeighborsRegressor,-0.273608,2.926463,13.717381,-0.273608,2.926463,13.717381,-0.273608,2.926463,13.717381
SVR,-0.065889,2.647575,11.480148,-0.065889,2.647575,11.480148,-0.065889,2.647575,11.480148
AdaBoostRegressor,-0.011625,2.648743,10.895693,-0.006461,2.664082,10.840079,-0.006161,2.649739,10.836844
BaggingRegressor,-0.181459,2.836277,12.724887,-0.167233,2.868073,12.571671,-0.091646,2.770194,11.757561
ExtraTreesRegressor,-0.055655,2.695789,11.369917,-0.054876,2.703595,11.361533,-0.037846,2.655031,11.178110
RandomForestRegressor,-0.064157,2.717108,11.461495,-0.056518,2.715448,11.379212,-0.077334,2.713504,11.603412
GradientBoostingRegressor,-0.135644,2.794743,12.231442,-0.129639,2.784249,12.166769,-0.130813,2.789223,12.179410
DecisionTreeRegressor,-0.190769,2.834345,12.825165,-0.151749,2.793782,12.404900,-0.161964,2.809747,12.514923


И так тоже R2 показывает плохие значения для идеи построения модели и выявления зависимостей